In [ ]:
# Make a better system naming things, with doucmentation
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
notebook_dir = os.path.abspath(os.getcwd())
src_path = os.path.join('/home/lerceg/LukaPosao/market_models/', 'src')
sys.path.append(src_path)

import market_models.src.models.predict_model as pm
import market_models.src.utils as utils

In [ ]:
data_name = 'netrisk_casco_2023_11_14__2023_11_20__2023_12_12__2023_12_22'
target_variable = 'ALLIANZ_price'
model_name = utils.get_model_name(data_name, target_variable)


data_path = utils.get_processed_data_path(data_name)
features_path = utils.get_features_path(data_name)
model_path = utils.get_model_path(model_name)


data, features = utils.load_data(data_path, features_path, target_variable = target_variable)
model = utils.load_model(model_path)

predictions = utils.predict(model, data[features])
errors = data[target_variable] - predictions
percentage_error = errors / data[target_variable] * 100

In [ ]:
def plotHistErrorPercenage(error : np.array) -> None:
    plt.hist(error, range = [error.min() - 1, error.max() + 1], bins=40,
             weights=np.ones(len(error)) / len(error))
    plt.xlabel('Error percentage')
    plt.ylabel('Percent of errors')
    plt.show()


In [ ]:
plotHistErrorPercenage(percentage_error)

In [ ]:
percentage_error.quantile([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99, 1])

In [ ]:
data[target_variable]

In [ ]:
data[data[target_variable] == 54900]

In [ ]:
data[(data['BonusMalus'] == 0) & (data['Age'].between(30, 40)) & (data['PostalCode'] < 2000)][['car_value', target_variable]].plot(kind = 'scatter', x = 'car_value', y = target_variable)

In [ ]:
data.plot(kind = 'scatter', x = 'car_value', y = target_variable)

In [ ]:
data.groupby(['car_value', 'Category', 'Age'])[target_variable].agg([('range', np.ptp), ('count', 'count'), ('min', 'min'), ('max', 'max'), ('minidx', 'idxmin'), ('maxidx', 'idxmax')])

In [ ]:
data.loc[[2923,2469]]

In [ ]:
1141680.0 / 574200.0


In [ ]:
gb_res = data.groupby(['Category', 'Age', 'kw', 'ccm', 'BonusMalus'])['car_value'].agg([('car_value_range', np.ptp), ('count', 'count'), ('idxmin', 'idxmin'), ('idxmax', 'idxmax')])

In [ ]:
def check(row):
    if row['idxmin'] == row['idxmax']:
        return 10
    try:
        min_row = data.loc[row['idxmin']]
        max_row = data.loc[row['idxmax']]
    
        price_ratio = max_row[target_variable] / min_row[target_variable]
        car_value_ratio = max_row['car_value'] / min_row['car_value']
        return abs(price_ratio - car_value_ratio)
    except Exception as e:
        return 100

In [ ]:
gb_res.apply(lambda x : check(x), axis = 1)

In [69]:
gb_res[gb_res['Catgory'] == 2]

KeyError: 'Catgory'